<a href="https://colab.research.google.com/github/shubhamnsut/BTP/blob/main/FN_LSTM_and_BidirectionalLSTM_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using Bidirectional LSTM



In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/BTP/Code/LSTM/Dataset/train.csv')

In [ ]:
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [ ]:
# Droping Nan Values from the Dataset
df=df.dropna()


In [ ]:
# Get the Independent Features from the Dataset
X=df.drop('label',axis=1)

In [ ]:
# Get the Dependent features from the Dataset
y=df['label']

In [ ]:
# there are 10361 news which are correct represnted by 0 
#there are 7924 news which are fake represnted by 1
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [ ]:
#Dataset with 18285 rows and 4 features
X.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.8.2'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [ ]:
### Vocabulary size
vocubalary_size=5000

### Onehot Representation

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
# Used for regular expression
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
fake_news = []
for i in range(0, len(messages)):
    print(i)
    news = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    news = news.lower()
    news = news.split()
    
    news = [ps.stem(word) for word in news if not word in stopwords.words('english')]
    news = ' '.join(news)
    fake_news.append(news)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [ ]:
fake_news

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
onehot=[one_hot(words,vocubalary_size)for words in fake_news] 
onehot

[[86, 2943, 1642, 4202, 916, 3294, 2235, 1559, 757, 3629],
 [3899, 4154, 875, 2824, 231, 4040, 1856],
 [4698, 4013, 3766, 4396],
 [4605, 735, 459, 1073, 1121, 4967],
 [4170, 231, 3010, 422, 4233, 3373, 231, 1885, 2643, 1845],
 [2420,
  4654,
  1304,
  1468,
  4384,
  3027,
  4593,
  730,
  3851,
  1162,
  3066,
  4561,
  3136,
  3953,
  1856],
 [1949, 3442, 1344, 1801, 1402, 2992, 4551, 3317, 3351, 1688, 2112],
 [689, 379, 1580, 3287, 3326, 3351, 3027, 3967, 3351, 1688, 2112],
 [2467, 1229, 2510, 1482, 3063, 3147, 4827, 3274, 3027, 276],
 [1575, 4148, 4502, 1997, 1324, 2508, 1378, 1081],
 [4014, 4186, 565, 1236, 4461, 2702, 2150, 602, 591, 1319, 4137],
 [1073, 2124, 916, 3147, 3027, 3326],
 [2370, 1926, 2111, 2017, 3632, 154, 2674, 1795, 4717],
 [4616, 4823, 3310, 4405, 266, 3302, 1403, 3351, 1688, 2112],
 [3454, 3240, 417, 618, 2701, 3351, 1688, 2112],
 [551, 2909, 2147, 2270, 1865, 1704, 4706, 3179, 2337, 3187],
 [2655, 1102, 4154],
 [2618, 776, 4273, 4372, 3027, 1746, 2373, 1856],
 

### Embedding Representation

In [ ]:
sent_length=20
embedded_document=pad_sequences(onehot,padding='pre',maxlen=sent_length)
print(embedded_document)

[[   0    0    0 ... 1559  757 3629]
 [   0    0    0 ...  231 4040 1856]
 [   0    0    0 ... 4013 3766 4396]
 ...
 [   0    0    0 ... 3351 1688 2112]
 [   0    0    0 ... 2959  168 2360]
 [   0    0    0 ... 4753 2723  956]]


In [ ]:
embedded_document[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   86,
       2943, 1642, 4202,  916, 3294, 2235, 1559,  757, 3629], dtype=int32)

For LSTM

In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocubalary_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


For Bidirectional LSTM

In [ ]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(vocubalary_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_document),y.shape

(18285, (18285,))

In [ ]:
import numpy as np
X_final=np.array(embedded_document)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [ ]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 11s 40ms/step - loss: 0.3161 - accuracy: 0.8453 - val_loss: 0.1958 - val_accuracy: 0.9183
Epoch 2/10
192/192 [==============================] - 7s 36ms/step - loss: 0.1357 - accuracy: 0.9473 - val_loss: 0.2273 - val_accuracy: 0.9200
Epoch 3/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0899 - accuracy: 0.9675 - val_loss: 0.2444 - val_accuracy: 0.9155
Epoch 4/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0613 - accuracy: 0.9784 - val_loss: 0.2574 - val_accuracy: 0.9155
Epoch 5/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0402 - accuracy: 0.9867 - val_loss: 0.3265 - val_accuracy: 0.9095
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0248 - accuracy: 0.9925 - val_loss: 0.3761 - val_accuracy: 0.9042
Epoch 7/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0215 - accuracy: 0.9935 - val_loss: 0.4296 - val_accuracy: 0.916

In [ ]:
y_pred1 = model1.predict(X_test)
y_pred1 = np.round(y_pred1).astype(int)


Bi-Directional LSTM

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred1)

array([[3105,  314],
       [ 234, 2382]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9091963545981773

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



Normal LSTM

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)


In [ ]:
confusion_matrix(y_test,y_pred)

array([[ 445, 2974],
       [ 629, 1987]])

In [ ]:
accuracy_score(y_test,y_pred)

0.4029826014913007

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.41      0.13      0.20      3419
           1       0.40      0.76      0.52      2616

    accuracy                           0.40      6035
   macro avg       0.41      0.44      0.36      6035
weighted avg       0.41      0.40      0.34      6035

